In [1]:
!pip install scipy==1.7.3
!pip install numpy==1.23.5
!pip install pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.1
    Uninstalling scipy-1.11.1:
      Successfully uninstalled scipy-1.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
chex 0.1.81 requires numpy>=1.25.0, but you have numpy 1.22.4 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pointpats 2.3.0 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.
py

In [2]:
import scipy
import numpy
import pandas
print(scipy.__version__)
print(numpy.__version__)
print(pandas.__version__)

1.7.3
1.23.5
1.5.3


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import os
import warnings
import statistics
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats
from scipy.stats import hmean,gmean
from sklearn.metrics import f1_score
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")

In [4]:
version_nb = 'V_ENSEMBLE'
f1_threshold = 0.5

In [5]:
df_v4 = pd.read_csv('/kaggle/input/fia-zindi-kp-v4/ensemble_all_v4_kaggle_1.5_raw.csv')
df_v5 = pd.read_csv('/kaggle/input/fia-zindi-kp-v5/ensemble_all_v5_kaggle_1.5_raw.csv')
df_v6 = pd.read_csv('/kaggle/input/fia-zindi-kp-v6/ensemble_all_v6_kaggle_1.5_raw.csv')
df_v7 = pd.read_csv('/kaggle/input/fia-zindi-kp-v7/ensemble_all_v7_kaggle_1.5_raw.csv')

print(df_v4.shape, df_v5.shape, df_v6.shape, df_v7.shape)


(1932, 2) (1932, 2) (1932, 2) (1932, 2)


In [6]:
df_ensemble = df_v4.set_index("ID").add_suffix("_v4").join(df_v5.set_index("ID").add_suffix("_v5")).join(df_v6.set_index("ID").add_suffix("_v6")).join(df_v7.set_index("ID").add_suffix("_v7"))
display(df_ensemble.corr())
vote = df_ensemble.apply(lambda x: hmean(x), axis=1)
df_ensemble['data_rate_t+1_trend'] = vote
df_ensemble = df_ensemble[['data_rate_t+1_trend']].copy()
df_ensemble['data_rate_t+1_trend'] = list(np.where(df_ensemble['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_ensemble = df_ensemble.reset_index()
df_ensemble.to_csv(f"ensemble_all_{version_nb}.csv", index = False)

,data_rate_t+1_trend_v4,data_rate_t+1_trend_v5,data_rate_t+1_trend_v6,data_rate_t+1_trend_v7
data_rate_t+1_trend_v4,1.000000,0.985494,0.947273,0.942511
data_rate_t+1_trend_v5,0.985494,1.000000,0.938918,0.935256
data_rate_t+1_trend_v6,0.947273,0.938918,1.000000,0.994967
data_rate_t+1_trend_v7,0.942511,0.935256,0.994967,1.000000


In [7]:
df_ensemble['data_rate_t+1_trend'].value_counts(normalize=True)

1    0.704969
0    0.295031
Name: data_rate_t+1_trend, dtype: float64